In [4]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter
import numpy as np
import itertools
import copy
import matplotlib.pyplot as plt
import json
import time
from dataraw_sampling import sample_queries, resolve_eval_expression, decode_key_to_vocab_token

In [14]:
num_attributes=6; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=4; multiple_OR_sets_bool=True
N_train=2048; N_val=2048; N_test=2048

expression_resolve_fn = resolve_eval_expression

sample_start_time = time.time()
data, stats = sample_queries(
    num_attributes, query_length_multiplier, nest_depth_int, multiple_OR_sets_bool, 
    expression_resolve_fn,
    N_train, N_val, N_test,
    debug=False)
sample_end_time = time.time()

print(f'Sample Time for {N_train+N_val+N_test} datapoints = {sample_end_time-sample_start_time} seconds')


save_start_time = time.time()
filename = '../Raw_Datasets/SET/{}Attr-{}Vals-{}LenMultiplier-{}NestDepth-{}Train-{}Val-{}Test.json'.format(
    num_attributes, num_attr_vals, query_length_multiplier, nest_depth_int, N_train, N_val, N_test)
with open(filename, 'w') as f:
    json.dump(data, f)
save_end_time = time.time()

filename = '../Raw_Datasets/SET/{}Attr-{}Vals-{}LenMultiplier-{}NestDepth-{}Train-{}Val-{}Test-Stats.json'.format(
    num_attributes, num_attr_vals, query_length_multiplier, nest_depth_int, N_train, N_val, N_test)
with open(filename, 'w') as f:
    json.dump(stats, f)
save_end_time = time.time()

print(f'Saving Time for {N_train+N_val+N_test} datapoints = {save_end_time-save_start_time} seconds')

Sample Time for 6144 datapoints = 138.9612045288086 seconds
Saving Time for 6144 datapoints = 29.92409658432007 seconds


In [2]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=9; nest_depth_int=1; multiple_OR_sets_bool=True
# N_train=1024; N_val=1024; N_test=1024

16e6 / 3072 * 31.317060947418213 / 3600

45.30824789846385

In [6]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=3; nest_depth_int=1; multiple_OR_sets_bool=True
# N_train=1024; N_val=1024; N_test=1024

16e6 / 3072 * 8.630649089813232 / 3600

12.486471484104792

In [8]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=1; multiple_OR_sets_bool=True
# N_train=1024; N_val=1024; N_test=1024

16e6 / 3072 * 5.308434247970581 / 3600

7.680026400420401

In [11]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=4; multiple_OR_sets_bool=True
# N_train=1024; N_val=1024; N_test=1024

16e6 / 3072 * 3.8778438568115234 / 3600

5.610306505803708

In [15]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=4; multiple_OR_sets_bool=True
# N_train=2048; N_val=2048; N_test=2048

16e6 / 3072 * 138.9612045288086 / 3600

201.04340932987353

In [ ]:
# num_attributes=6; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=4; multiple_OR_sets_bool=True
# N_train=2048; N_val=2048; N_test=2048

16e6 / 3072 * 7.696515083312988 / 3600

In [ ]:
# num_attributes=4; num_attr_vals=3; nest_depth = 4; num_cards multiple 2
16000000 / 1000 * 1.32326340675354 / 3600